# Chap 10 데이터 집계와 그룹 연산

 
* 하나 이상의 를 이용해서 pandas 객체를 여러 조각으로 나누는 법

* 합계, 평균, 표준편차, 사용자 정의 함수 같은 그룹 요약 통계 계산법

* 정규화, 선형회귀, 등급 또는 부분 집합 선택 등 집단 내 변형이나 다른 조작을 적용하는 방법

* 피벗테이블과 교차일람표를 구하는 방법

* 변위치 분석과 다른 통계 집단 분석을 수행하는 방법


## GroupBy 메카닉 


1. 분리 
 - 하나의 Key를 기준으로 **분리**
 
2. 적용
 - 함수를 나뉘어진 그룹에 **적용**시켜 새로운 값 얻어내기

3.  결합
 - 함수가 적용된 결과를 하나의 객체로 **결합**
 
 
---------------------------------

Google의 논문 [MapReduce: Simplified Data Processing on Large Clusters](https://research.google/pubs/pub62/)에 영향받은

데이터 분석에서는 Hadley Wickhan의 논문 [The Split-Apply-Combine Strategy for Data Analysis](https://www.jstatsoft.org/v40/i01/paper)

에서 시작되었음
 
------------------------------ 
> 두 논문 다 시간 날 때 읽어 보시길 권합니다..!

In [10]:
import numpy as np
import pandas as pd

In [19]:
df = pd.DataFrame(
    {
        "key1": ["a", "a", "b", "b", "a"],
        "key2": ["one", "two", "one", "two", np.nan],
        "data1": np.random.randn(5),
        "data2": np.random.randn(5),
    }
)
df

,key1,key2,data1,data2
0,a,one,1.532423,-0.748299
1,a,two,0.923407,-0.616324
2,b,one,0.262773,-2.045550
3,b,two,1.645197,-0.132051
4,a,NaN,-0.417500,0.184773


In [20]:
grouped = df["data1"].groupby(df["key1"])
grouped  # 단순히 groupby만 쓰면 groupby 객체만 나옵니다.

In [22]:
grouped.mean()  # key1 칼럼 기준으로 묶어서 평균을 계산해주었음.
# 새로운 이 Series 객체의 색인은 우리가 groupby로 지정한 'key1임'

key1
a    0.679443
b    0.953985
Name: data1, dtype: float64

In [23]:
means = (
    df["data1"].groupby([df["key1"], df["key2"]]).mean()
)  # groupby key를 key1, key2 두개를 줬기에 계층적 index를 갖게됨
means

key1  key2
a     one     1.532423
      two     0.923407
b     one     0.262773
      two     1.645197
Name: data1, dtype: float64

In [24]:
means.unstack()  # unstack으로 풀어버리기도 가능

key2,one,two
key1,,
a,1.532423,0.923407
b,0.262773,1.645197


In [25]:
states = np.array(["Ohio", "California", "California", "Ohio", "Ohio"])
years = np.array([2005, 2005, 2006, 2005, 2006])
df["data1"].groupby([states, years]).mean()  # 기존 data1에 없던 index를 넣어서도 가능함.

California  2005    0.923407
            2006    0.262773
Ohio        2005    1.588810
            2006   -0.417500
Name: data1, dtype: float64

In [26]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    one   1.532423 -0.748299
     two   0.923407 -0.616324
b    one   0.262773 -2.045550
     two   1.645197 -0.132051

In [27]:
df.groupby(
    "key1"
).mean()  # 위와 비교하면 key2가 빠져있는데 key2는 숫자가 아니라 (범주형 데이터라) mean 계산이 불가능하기 때문
# 이런 칼럼을 성가신 칼럼 nuisance column이라 부르고 결과에서 제외함.

,data1,data2
key1,,
a,0.679443,-0.393283
b,0.953985,-1.088800


In [28]:
df.groupby(["key1", "key2"]).size()  # 이때 nan은 제외힘

key1  key2
a     one     1
      two     1
b     one     1
      two     1
dtype: int64

## 그룹간 순회하기

In [46]:
for name, group in df.groupby("key1"): # 하나씩 for 문으로 받기도 가능. 뭔가 각 종류별로 데이터를 넣을 때 사용할 수 있을 듯 `
    print(name)
    print('-'* 10)
    print(group)       
    print('\n')

a
----------
  key1 key2     data1     data2
0    a  one  1.532423 -0.748299
1    a  two  0.923407 -0.616324
4    a  NaN -0.417500  0.184773


b
----------
  key1 key2     data1     data2
2    b  one  0.262773 -2.045550
3    b  two  1.645197 -0.132051




In [45]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print('-' * 10)
    print(group)
    print('\n')

('a', 'one')
----------
  key1 key2     data1     data2
0    a  one  1.532423 -0.748299


('a', 'two')
----------
  key1 key2     data1     data2
1    a  two  0.923407 -0.616324


('b', 'one')
----------
  key1 key2     data1    data2
2    b  one  0.262773 -2.04555


('b', 'two')
----------
  key1 key2     data1     data2
3    b  two  1.645197 -0.132051




In [36]:
pieces = dict(list(df.groupby('key1')))   # dic 형식으로 받기 도 가능 
pieces             

{'a':   key1 key2     data1     data2
 0    a  one  1.532423 -0.748299
 1    a  two  0.923407 -0.616324
 4    a  NaN -0.417500  0.184773,
 'b':   key1 key2     data1     data2
 2    b  one  0.262773 -2.045550
 3    b  two  1.645197 -0.132051}

In [41]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [50]:
grouped = df.groupby(df.dtypes, axis=1)  # 행을 기준이 아닌 열(axis = 1)을 기준으로도 그룹을 만들 수 있음 
for dtype, group in grouped:
    print(dtype)
    print('-'* 20)
    print(group)
    print('\n')

float64
--------------------
      data1     data2
0  1.532423 -0.748299
1  0.923407 -0.616324
2  0.262773 -2.045550
3  1.645197 -0.132051
4 -0.417500  0.184773


object
--------------------
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  NaN




## 컬럼이나 컬럼의 일부만 선택하기 
생략

## dict과 Series에서 그룹핑하기